In [1]:
import json
import re
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB

In [2]:
nl_f = open('../data/nl.json')
en_f = open('../data/en.json')
nl_data = json.load(nl_f)
en_data = json.load(en_f)

In [3]:
urls_tokenized = []
excluded_keywords = ['www', 'index', 'html', 'htm', 'html', 'http', 'https']
nl_tlds = ['.nl/', '.be/', '.su/', '.aw/', '.sx/', '.cw/']
other_tlds = ['.com/', '.net/', '.org/']

for element in nl_data:
    url_element = element['siteUrl'] + ('' if element['siteUrl'].endswith('/') else '/')
    split = [word for word in re.split('[^a-zA-Z]', url_element) 
        if len(word) >= 2 and word not in excluded_keywords]
    custom_feat =[
        1 if any(tld in url_element for tld in nl_tlds) else 0,
        1 if any(tld in url_element for tld in other_tlds) else 0
    ]
    urls_tokenized.append([split, custom_feat, url_element, 1])

for element in en_data:
    url_element = element['siteUrl'] + ('' if element['siteUrl'].endswith('/') else '/')
    split = [word for word in re.split('[^a-zA-Z]', url_element) 
        if len(word) >= 2 and word not in excluded_keywords]
    custom_feat =[
        1 if any(tld in url_element for tld in nl_tlds) else 0,
        1 if any(tld in url_element for tld in other_tlds) else 0
    ]
    urls_tokenized.append([split, custom_feat, url_element, 0])

In [4]:
print(urls_tokenized[0])

[['tevansleen', 'nl'], [1, 0], 'https://www.tevansleen.nl/', 1]


In [5]:
# NOTE: NOT NECESSARY SINCE TRAIN_TEST_SPLIT ALREADY SHUFFLES VALUES

#Train Test split - ONLY RUN ONCE AFTER INITIALIZATION
# random.shuffle(urls_tokenized)

In [6]:
X_tokens, X_url, y = [], [], []
for url in urls_tokenized:
    X_tokens.append(url[0])
    X_url.append(url[2])
    y.append(url[3])

print(X_tokens[0], y[0])

vectorizer = CountVectorizer()
X_tokens = vectorizer.fit_transform(X_url)

print(X_tokens)

['tevansleen', 'nl'] 1
  (0, 48496)	1
  (0, 113266)	1
  (0, 97817)	1
  (0, 70871)	1
  (1, 113266)	1
  (1, 70871)	1
  (1, 48494)	1
  (1, 24724)	1
  (2, 48496)	1
  (2, 70871)	1
  (2, 49062)	1
  (3, 48496)	1
  (3, 113266)	1
  (3, 70871)	1
  (3, 111964)	1
  (3, 90715)	1
  (4, 48496)	1
  (4, 113266)	1
  (4, 70871)	1
  (4, 95156)	1
  (5, 48496)	1
  (5, 113266)	1
  (5, 70871)	1
  (5, 100724)	1
  (6, 48496)	1
  :	:
  (108457, 48494)	1
  (108457, 102419)	1
  (108457, 108915)	1
  (108458, 48496)	1
  (108458, 113266)	1
  (108458, 74432)	1
  (108458, 104923)	1
  (108459, 48496)	1
  (108459, 24700)	1
  (108459, 47512)	1
  (108459, 90435)	1
  (108459, 90465)	1
  (108459, 42812)	1
  (108459, 101399)	1
  (108460, 113266)	1
  (108460, 48494)	1
  (108460, 24700)	1
  (108460, 23514)	1
  (108460, 87127)	1
  (108460, 100176)	1
  (108460, 43455)	1
  (108461, 48496)	1
  (108461, 113266)	1
  (108461, 24700)	1
  (108461, 54059)	1


In [7]:
print(vectorizer.get_feature_names_out()[80000])

primhome


In [8]:
X_token_train, X_token_test, y_token_train, y_token_test = train_test_split(X_tokens, y, test_size=0.2)

In [9]:
# ccTLD - no training necessary because it just looks at TLD:
ccTLD_y = []
for url in X_url:
    ccTLD_y.append(1 if any(tld in url for tld in nl_tlds) else 0)

print(len(ccTLD_y))

# ccTLD+ Would give the same outcome as TLD in our case because it counts generic TLD's (.com/.net/.org) as non-dutch anyway.

108462


In [10]:
print(X_token_train[0])
print(y_token_train[0])

  (0, 48496)	1
  (0, 113266)	1
  (0, 48468)	1
  (0, 74432)	1
  (0, 3537)	1
  (0, 31195)	1
  (0, 51312)	1
  (0, 79482)	1
  (0, 21699)	1
0


In [11]:
# Token features
gnb = GaussianNB()
y_pred = gnb.fit(X_token_train.toarray(), y_token_train).predict(X_token_test.toarray())

In [ ]:
# Custom features

In [ ]:
confusion_matrix(y, ccTLD_y)

In [ ]:
print(y_pred)

Error: Session cannot generate requests

In [ ]:
confusion_matrix(y_token_test, y_pred)

Error: Session cannot generate requests